In [19]:
import datasets
# imports
import evaluate
import numpy as np
from numpy.core import overrides
from datasets import load_dataset, Dataset
from datasets import Features
from datasets import ClassLabel
from datasets import Value
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig, AutoModelForSequenceClassification, \
    TrainingArguments, Trainer
import torch

In [20]:
RES_PATH = os.path.abspath('../resources/data/') + '/'


In [21]:
# load data into Series (retrieving train, test and optimization sets)
with open(RES_PATH + 'laptop/train.json', 'r', encoding='utf-8') as file_train_laptop:
    raw_train_laptop = pd.read_json(file_train_laptop, encoding='utf-8', orient='index').fillna("")

with open(RES_PATH + 'laptop/test.json', 'r', encoding='utf-8') as file_test_laptop:
    raw_test_laptop = pd.read_json(file_test_laptop, encoding='utf-8', orient='index').fillna("")

with open(RES_PATH + 'laptop/dev.json', 'r', encoding='utf-8') as file_dev_laptop:
    raw_dev_laptop = pd.read_json(file_dev_laptop, encoding='utf-8', orient='index').fillna("")

with open(RES_PATH + 'rest/train.json', 'r', encoding='utf-8') as file_train_res:
    raw_train_res = pd.read_json(file_train_res, encoding='utf-8', orient='index').fillna("")

with open(RES_PATH + 'rest/test.json', 'r', encoding='utf-8') as file_test_res:
    raw_test_res = pd.read_json(file_test_res, encoding='utf-8', orient='index').fillna("")

with open(RES_PATH + 'rest/dev.json', 'r', encoding='utf-8') as file_dev_res:
    raw_dev_res = pd.read_json(file_dev_res, encoding='utf-8', orient='index').fillna("")

raw_train_laptop



,label,sentence
0,"[B, O, O, O, O, O, O, B, I, O, O, O, O, O]","[Keyboard, is, great, but, primary, and, secon..."
1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[I, bought, this, laptop, about, a, month, ago..."
2,"[O, O, O, O, O, O, O, O, O, O, O, O]","[I, am, however, pleased, that, it, is, still,..."
3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[I, went, to, my, local, Best, Buy, looking, f..."
4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[The, Apple, MC371LL/, A, 2.4Ghz, 15.4-, inch,..."
...,...,...
2890,"[O, O, O, O, O, O, O, O, B, I, O, O, O, O, O, ...","[After, talking, it, over, with, the, very, kn..."
2891,"[O, B, I, O, O, O, O, O, O, O, O, O, O, O, O, ...","[If, internet, connectivity, is, important, I,..."
2892,"[O, O, O, O, O, O, O, O, B, O, O, O, O, O, O, ...","[My, friend, just, had, to, replace, his, enti..."
2893,"[O, O, O, O, O, O, O, O, O, O, O, O, O]","[I, work, with, kids, and, they, love, making,..."


In [22]:
# preprocessing

# [The, Apple, MC371LL/, A, 2.4Ghz, 15.4-, inch, MacBook, Pro, Notebook, is, a, horrible, waste, of, money, .]
# -> [The, Apple MC371LL/A 2.4Ghz 15.4-inch MacBook Pro Notebook, is, a, horrible, waste, of, money, .]

# [Did, n't, expect, to, repair, it, at, once, !]
# -> [Didn't, expect, to, repair, it, at, once, !]

In [23]:
# data reduction:
# convert data to lowercase, correct spelling, remove/handle stopwords, remove/handle punctuation, remove/handle numbers, remove/handle special characters
# detect ambiguous words and re-label them correctly

In [24]:
# data transformation:
# unnecessary for this project, data already in the correct format: string and integer values are suitable for the model

In [25]:
# data cleaning:
# remove too short/long sentences, remove duplicate sentences, remove empty sentences, remove sentences with only stopwords, remove sentences with only punctuation, remove sentences with only numbers, remove sentences with only special characters

In [26]:
# data integration:
# unnecessary for this project, only one datasource present

In [27]:
# modeling
# https://www.geeksforgeeks.org/hidden-markov-model-in-machine-learning/

# modeling (Co-Pilot suggestions LOL):
# model selection: Naive Bayes, SVM, Decision Tree, Random Forest, Logistic Regression, Neural Network, RNN, LSTM, BERT
# model training: 70/10/20 split, 5-fold cross-validation
# model optimization: hyperparameter tuning, cross-validation, grid search, random search
# model testing:


In [28]:
ds_train_laptop = Dataset.from_pandas(raw_train_laptop)
ds_test_laptop = Dataset.from_pandas(raw_test_laptop)
ds_dev_laptop = Dataset.from_pandas(raw_dev_laptop)

ds_train_res = Dataset.from_pandas(raw_train_res)
ds_test_res = Dataset.from_pandas(raw_test_res)
ds_dev_res = Dataset.from_pandas(raw_dev_res)

laptop_dataset_dict = datasets.DatasetDict({
    "train": ds_train_laptop,
    "test": ds_test_laptop,
    "dev": ds_dev_laptop
})

res_dataset_dict = datasets.DatasetDict({
    "train": ds_train_res,
    "test": ds_test_res,
    "dev": ds_dev_res
})

In [29]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [30]:
# define function to tokenize data (sentence column) with BERT tokenizer
def tokenize_data(dataset_dict):
    return tokenizer(
        dataset_dict['sentence'],
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        is_split_into_words=True)

# tokenize data with defined function
laptop_dataset_dict_enc = laptop_dataset_dict.map(tokenize_data, batched=True)
res_dataset_dict_enc = res_dataset_dict.map(tokenize_data, batched=True)

# set format to torch
laptop_dataset_dict_enc.set_format("torch")
res_dataset_dict_enc.set_format("torch")

Map:   0%|          | 0/2895 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/1850 [00:00<?, ? examples/s]

Map:   0%|          | 0/676 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [31]:
laptop_dataset_dict_enc["dev"]


Dataset({
    features: ['label', 'sentence', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 150
})

In [32]:
clf_metrics = evaluate.combine(["accuracy", 'f1', "precision", "recall"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels)


args = TrainingArguments(
    output_dir="bert-base-uncased_auto-seq-model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=laptop_dataset_dict["train"],
    eval_dataset=laptop_dataset_dict["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

In [33]:
trainer.args.device


device(type='cpu')

In [34]:
trainer.train()


C:\Users\jannik\programming_projects\nlp-project-ae\venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [ ]:
trainer.evaluate()


In [ ]:
trainer.evaluate(laptop_dataset_dict["dev"])

In [ ]:
# model evaluation with metrics precision, recall, accuracy, f1-score and confusion matrix
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html